In [1]:
# Environment: python3
# Developer: EP

In [2]:
# Algo:
# Batch program to update Average Cost  
# Scope: Single cost centre, Single Item
#        Exclude BOM Avergae Cost Calculation
#        Exclud Manufactured Goods algo
#
# 1. Initiatization, Input Processing
#    Read File:  TRX, DAY_WAC, BOM, ITEM for BOD & ITEM
#    Input BOD
#    Input ITEM
#    Input the BOD date & select BOD details (Avg Cost, Qty...etc) & EOD details
#    
# 2. Calculate & Reset BOD Average Cost if BOD Qty = 0
#    Select Actual Cost Transactions (GD_IN & GD_RETN)
#    Sort by Date & Time
#    Using Actual Cost, Calculate EOD Interim Average Cost, Total Qty
#    If BOD Average Cost = 0 or Qty = 0.
#    If previous day EOD Average Cost <> 0
#        BOD Average Cost = previous day EOD Average Cost
#    else
#        BOD Average Cost = Standard Cost in Recipe
#        
# 3. Process transactions using Actual Cost (GD_PO, GD RETN, INV_COST_QTY)
#    GD_PO: Purchased; GD_RETN: Returned, GD_IN: Transfered in; RCP_OUT:Manufactured Out
#    Sort by Date and Time
#    Calculate EOD Interim Average Cost, Total Qty
#    
# 4. Process transactions using BOD Average Cost (GD_OUT, RCP_OUT, INV_COST_QTY)
#    Sort by Date and Time
#    Use BOD Average cost for these transactions
#    Calculate EOD Interum Average Cost, Total Qty (Prior to COGS)
#    ** Note that this re-assigning of BOD Average cost is assuming that the original input program
#       is unable to or incorrectly calculate the Average Cost at the time of GD_OUT, RCP_OUT.
#       Otherwise, ideally, it should be done at transaction entry time & worse case used Standard Cost
#       if the Inventory Qty or Average Cost = 0 or unknown.
#
# 5. Process COGS transactions using EOD Interim Average Cost
#    Sort by Date and Time
#    Using Average Cost, calculate EOD Interum Average Cost, Total Qty#
#
# 6. Calucate sum of PO, Inventory Transfer, Inventory Adjustment, Manufacturing and COGS fields
#
# 7.  Process Journals (DR & CR Accounts) into transactions records
#
# 8. Write into file, save() close()

In [3]:
import pandas as pd
import numpy as np

In [4]:
path = r'/Users/edi/Desktop/MyPython/WAC/WAC.xlsx'
path_eod = r'/Users/edi/Desktop/MyPython/WAC/WAC_EOD.xlsx'
writer = pd.ExcelWriter(path_eod, engine = 'xlsxwriter')

In [5]:
df = pd.read_excel (path, sheet_name='TRX')
dfwac = pd.read_excel (path, sheet_name='DAY_WAC')
dfbom = pd.read_excel (path, sheet_name='BOM')
dfitem = pd.read_excel (path, sheet_name='ITEM')

In [6]:
bod_date = '2020-05-02'
sel_item = 'A'

In [7]:
dfbod = dfwac[(dfwac['DATE'] == bod_date) & (dfwac['ITEM']== sel_item)]
bod_idx = dfbod.index.values[0]
bod_avg_cost = dfbod.at[bod_idx,'BOD_AVG_COST']
bod_qty = dfbod.at[bod_idx,'BOD_QTY']
bod_trx_amt = dfbod.at[bod_idx,'BOD_TRX_AMT']

In [8]:
dfitem = dfitem[(dfitem['ITEM'] == sel_item)]
item_idx = dfitem.index.values[0]
std_cost = dfitem['STD_COST'].values[0]

In [9]:
# Calculate Average Cost from DataFrame
# t_qty, t_trx_amt, t_avg_cost, t_amt_avg_cost = trx_cal(dftrx) 

In [10]:
def trx_cal (f_df):
    f_qty = f_df['QTY'].sum()
    f_trx_amt = f_df['TRX_AMT'].sum()
    f_avg_cost = round((f_trx_amt / f_qty) , 2)
    f_amt_avg_cost = round((f_qty * f_avg_cost) , 2)
    return (f_qty, f_trx_amt, f_avg_cost, f_amt_avg_cost)

In [11]:
df

,ITEM,DATE,TIME,DOC_TYPE,DOC_NO,RMKS,QTY,ACT_U_COST,USE_AVG_COST,AVG_COST,ACT_AMT,AMT_AVG_COST,TRX_AMT,AMT_RD_VAR,DR_ACCT,CR_ACCT,DR_COST_C,CR_COST_C
0,A,2020-05-02,11:00:00,GD_PO,1,Goods Received,2,11.0,0.0,NAN,22.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN
1,A,2020-05-02,11:30:00,GD_PO,2,Goods Received,1,12.0,0.0,NAN,12.0,0.0,12.0,NaN,NaN,NaN,NaN,NaN
2,A,2020-05-02,13:00:00,GD_RETN,3,Goods Returned for 2,-1,11.0,0.0,NAN,-11.0,0.0,-11.0,NaN,NaN,NaN,NaN,NaN
3,A,2020-05-02,12:00:00,GD_OUT,4,Goods Transferred Out,-2,NaN,1.0,10,NaN,-20.0,-20.0,NaN,NaN,NaN,NaN,NaN
4,A,2020-05-02,12:00:00,GD_IN,5,Goods Transferred In,1,10.5,1.0,NaN,10.5,0.0,10.5,NaN,NaN,NaN,NaN,NaN
5,A,2020-05-02,19:00:00,RCP_OUT,6,Ingredients used for Recipe DOC 107 ***,-2,10.0,1.0,NaN,NaN,0.0,-20.0,NaN,NaN,NaN,NaN,NaN
6,A,2020-05-02,14:00:00,INV_QTY_ADJ,7,Inventory Qty Adjustment (e.g. write-off),-1,NaN,1.0,10,NaN,-10.0,-10.0,NaN,NaN,NaN,NaN,NaN
7,A,2020-05-02,15:00:00,INV_COST_ADJ,8,Inventory Cost Adjustment,0,NaN,0.0,NaN,-5.0,0.0,-5.0,NaN,NaN,NaN,NaN,NaN
8,A,2020-05-02,15:00:00,COGS,10,Goods sold,-5,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
9,B,2020-05-02,11:00:00,GD_PO,102,Goods Received,2,16.0,0.0,NaN,22.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN


In [12]:
dftrx = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['GD_PO','GD_RETN','GD_IN','RCP_OUT','INV_COST_ADJ']))]

In [13]:
# Reset Begin of Day Average Cost if Begin of Day Qty = 0
#   Use Current Day Actual Cost Transactions to derive the Average Cost
#   If there are no trasnactions, then use Standard Cost

In [14]:
dftrx

,ITEM,DATE,TIME,DOC_TYPE,DOC_NO,RMKS,QTY,ACT_U_COST,USE_AVG_COST,AVG_COST,ACT_AMT,AMT_AVG_COST,TRX_AMT,AMT_RD_VAR,DR_ACCT,CR_ACCT,DR_COST_C,CR_COST_C
0,A,2020-05-02,11:00:00,GD_PO,1,Goods Received,2,11.0,0.0,NAN,22.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN
1,A,2020-05-02,11:30:00,GD_PO,2,Goods Received,1,12.0,0.0,NAN,12.0,0.0,12.0,NaN,NaN,NaN,NaN,NaN
2,A,2020-05-02,13:00:00,GD_RETN,3,Goods Returned for 2,-1,11.0,0.0,NAN,-11.0,0.0,-11.0,NaN,NaN,NaN,NaN,NaN
4,A,2020-05-02,12:00:00,GD_IN,5,Goods Transferred In,1,10.5,1.0,NaN,10.5,0.0,10.5,NaN,NaN,NaN,NaN,NaN
5,A,2020-05-02,19:00:00,RCP_OUT,6,Ingredients used for Recipe DOC 107 ***,-2,10.0,1.0,NaN,NaN,0.0,-20.0,NaN,NaN,NaN,NaN,NaN
7,A,2020-05-02,15:00:00,INV_COST_ADJ,8,Inventory Cost Adjustment,0,NaN,0.0,NaN,-5.0,0.0,-5.0,NaN,NaN,NaN,NaN,NaN


In [15]:
dfbod

,ITEM,DATE,TIME,BOD_QTY,BOD_AVG_COST,BOD_STD_COST,BOD_AMT_AVG_COST,BOD_TRX_AMT,BOD_BOM_COST,PO_QTY,...,MFG_QTY,MFG_AMT,COGS_QTY,COGS_AMT,EOD_QTY,EOD_AVG_COST,EOD_STD_COST,EOD_AMT_AVG_COST,EOD_TRX_AMT,EOD_BOM_COST
1,A,2020-05-02,00:00:01,10.0,10.0,10.2,100.0,100.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
if bod_qty == 0 :
    dfbod.at[bod_idx,'AMT_AVG_COST'] = 0
    if dftrx.empty:
        bod_avg_cost = std_cost
        dfbod.at[bod_idx,'AVG_COST'] = std_cost   
    else:
        t_qty, t_trx_amt, t_avg_cost, t_amt_avg_cost = trx_cal(dftrx)       
        bod_avg_cost = t_avg_cost
        dfbod.at[bod_idx,'AVG_COST'] = bod_avg_cost

In [17]:
dfbod

,ITEM,DATE,TIME,BOD_QTY,BOD_AVG_COST,BOD_STD_COST,BOD_AMT_AVG_COST,BOD_TRX_AMT,BOD_BOM_COST,PO_QTY,...,MFG_QTY,MFG_AMT,COGS_QTY,COGS_AMT,EOD_QTY,EOD_AVG_COST,EOD_STD_COST,EOD_AMT_AVG_COST,EOD_TRX_AMT,EOD_BOM_COST
1,A,2020-05-02,00:00:01,10.0,10.0,10.2,100.0,100.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Process GD_PO , GD_RETN , RCP_OUT, INV_COST_ADJ
# Use the Transaction Amount (not the Average Cost Amount)

In [19]:
if not dftrx.empty:
    t_qty, t_trx_amt, t_avg_cost, t_amt_avg_cost = trx_cal(dftrx) 
    eod_qty = t_qty + bod_qty
    eod_trx_amt = t_trx_amt + bod_trx_amt
else:
    eod_qty = bod_qty
    eod_trx_amt = bod_trx_amt

eod_avg_cost = round((eod_trx_amt / eod_qty) , 2)
eod_amt_avg_cost = round((eod_avg_cost * eod_qty) , 2)

In [20]:
dfbod.at[bod_idx,'EOD_QTY'] = eod_qty
dfbod.at[bod_idx,'EOD_AVG_COST'] = eod_avg_cost
dfbod.at[bod_idx,'EOD_STD_COST'] = std_cost
dfbod.at[bod_idx,'EOD_AMT_AVG_COST'] = eod_amt_avg_cost
dfbod.at[bod_idx,'EOD_TRX_AMT'] = eod_trx_amt

In [21]:
print (eod_avg_cost)
print (eod_qty)

9.86
11.0


In [22]:
# Ensure that Kitchen Orders transactions RCP_OUT are created
# RCP_OUT is only for Manufactured Goods resulting in new items
# Example Item A is used to make Item B
# 
# RCP will not be used in the case where Item A used for Sale of Item A, this will be taken care of by COGS
# A more advanced feature will be to compare the the Kitchen Orders vs. COGS; 
# which is outside the scope of this program.
# Will address this Alogo in another program to cater which will be useful to track Buffet consumption vs. items made

In [23]:
# Calculate the rest of the transactions that are using BOD Average Cost
# Select 'GD_OUT' , 'INV_QTY_ADJ' 
# Assign BOD Average Cost to these transactions 
#
# Caluclate EOD Average cost and EOD TRX_AMT (prior to COGS)

In [24]:
dftrx2 = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['GD_OUT','INV_QTY_ADJ']))]

In [25]:
dftrx2

,ITEM,DATE,TIME,DOC_TYPE,DOC_NO,RMKS,QTY,ACT_U_COST,USE_AVG_COST,AVG_COST,ACT_AMT,AMT_AVG_COST,TRX_AMT,AMT_RD_VAR,DR_ACCT,CR_ACCT,DR_COST_C,CR_COST_C
3,A,2020-05-02,12:00:00,GD_OUT,4,Goods Transferred Out,-2,NaN,1.0,10,NaN,-20.0,-20.0,NaN,NaN,NaN,NaN,NaN
6,A,2020-05-02,14:00:00,INV_QTY_ADJ,7,Inventory Qty Adjustment (e.g. write-off),-1,NaN,1.0,10,NaN,-10.0,-10.0,NaN,NaN,NaN,NaN,NaN


In [26]:
# Update transactions with average cost

In [27]:
if not dftrx2.empty:
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['GD_OUT','RCP_OUT','INV_QTY_ADJ'])) , 'AVG_COST'] = bod_avg_cost 
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['GD_OUT','RCP_OUT','INV_QTY_ADJ'])) , 'AMT_AVG_COST'] = (df['QTY'] * bod_avg_cost)
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['GD_OUT','RCP_OUT','INV_QTY_ADJ'])) , 'TRX_AMT'] = (df['QTY'] * bod_avg_cost)

In [28]:
df

,ITEM,DATE,TIME,DOC_TYPE,DOC_NO,RMKS,QTY,ACT_U_COST,USE_AVG_COST,AVG_COST,ACT_AMT,AMT_AVG_COST,TRX_AMT,AMT_RD_VAR,DR_ACCT,CR_ACCT,DR_COST_C,CR_COST_C
0,A,2020-05-02,11:00:00,GD_PO,1,Goods Received,2,11.0,0.0,NAN,22.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN
1,A,2020-05-02,11:30:00,GD_PO,2,Goods Received,1,12.0,0.0,NAN,12.0,0.0,12.0,NaN,NaN,NaN,NaN,NaN
2,A,2020-05-02,13:00:00,GD_RETN,3,Goods Returned for 2,-1,11.0,0.0,NAN,-11.0,0.0,-11.0,NaN,NaN,NaN,NaN,NaN
3,A,2020-05-02,12:00:00,GD_OUT,4,Goods Transferred Out,-2,NaN,1.0,10,NaN,-20.0,-20.0,NaN,NaN,NaN,NaN,NaN
4,A,2020-05-02,12:00:00,GD_IN,5,Goods Transferred In,1,10.5,1.0,NaN,10.5,0.0,10.5,NaN,NaN,NaN,NaN,NaN
5,A,2020-05-02,19:00:00,RCP_OUT,6,Ingredients used for Recipe DOC 107 ***,-2,10.0,1.0,10,NaN,-20.0,-20.0,NaN,NaN,NaN,NaN,NaN
6,A,2020-05-02,14:00:00,INV_QTY_ADJ,7,Inventory Qty Adjustment (e.g. write-off),-1,NaN,1.0,10,NaN,-10.0,-10.0,NaN,NaN,NaN,NaN,NaN
7,A,2020-05-02,15:00:00,INV_COST_ADJ,8,Inventory Cost Adjustment,0,NaN,0.0,NaN,-5.0,0.0,-5.0,NaN,NaN,NaN,NaN,NaN
8,A,2020-05-02,15:00:00,COGS,10,Goods sold,-5,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
9,B,2020-05-02,11:00:00,GD_PO,102,Goods Received,2,16.0,0.0,NaN,22.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN


In [29]:
# Update EOD record 

In [30]:
if not dftrx2.empty:
    eod_qty = dftrx2['QTY'].sum() + eod_qty
    eod_trx_amt = dftrx2['TRX_AMT'].sum() + eod_trx_amt
    eod_avg_cost = round((eod_trx_amt / eod_qty), 2)
    eod_amt_avg_cost = eod_avg_cost * eod_qty

In [31]:
dfbod.at[bod_idx,'EOD_QTY'] = eod_qty
dfbod.at[bod_idx,'EOD_AVG_COST'] = eod_avg_cost
dfbod.at[bod_idx,'EOD_STD_COST'] = std_cost
dfbod.at[bod_idx,'EOD_AMT_AVG_COST'] = eod_amt_avg_cost
dfbod.at[bod_idx,'EOD_TRX_AMT'] = eod_trx_amt

In [32]:
print (eod_avg_cost)
print (eod_qty)

9.81
8.0


In [33]:
# Process Cost of Goods Sold based on the EOD Average Cost

In [34]:
dftrx3 = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['COGS']))]

In [35]:
if not dftrx3.empty:
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['COGS'])) , 'AVG_COST'] = eod_avg_cost 
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['COGS'])) , 'AMT_AVG_COST'] = (df['QTY'] * eod_avg_cost)
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['COGS'])) , 'TRX_AMT'] = (df['QTY'] * eod_avg_cost)

In [36]:
# Refresh View of transaction DataFrame because Source (df) has been updated

In [37]:
dftrx3 = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['COGS']))]

In [38]:
if not dftrx3.empty:
    eod_qty = eod_qty + (dftrx3['QTY'].sum())
    eod_trx_amt = eod_trx_amt  + (dftrx3['TRX_AMT'].sum()) 
    eod_amt_avg_cost = eod_avg_cost * eod_qty

In [39]:
dftrx3

,ITEM,DATE,TIME,DOC_TYPE,DOC_NO,RMKS,QTY,ACT_U_COST,USE_AVG_COST,AVG_COST,ACT_AMT,AMT_AVG_COST,TRX_AMT,AMT_RD_VAR,DR_ACCT,CR_ACCT,DR_COST_C,CR_COST_C
8,A,2020-05-02,15:00:00,COGS,10,Goods sold,-5,NaN,NaN,9.81,NaN,-49.05,-49.05,NaN,NaN,NaN,NaN,NaN


In [40]:
print (eod_avg_cost)
print (eod_qty)

9.81
3.0


In [41]:
# Update BOD/EOD Record with EOD details

In [42]:
dfwac['EOD_QTY'] = dfwac['EOD_QTY'].astype(float)
dfwac['EOD_AVG_COST'] = dfwac['EOD_AVG_COST'].astype(float)
dfwac['EOD_AMT_AVG_COST'] = dfwac['EOD_AMT_AVG_COST'].astype(float)
dfwac['EOD_TRX_AMT'] = dfwac['EOD_TRX_AMT'].astype(float)

In [43]:
dfbod.at[bod_idx,'EOD_QTY'] = eod_qty
dfbod.at[bod_idx,'EOD_AVG_COST'] = eod_avg_cost
dfbod.at[bod_idx,'EOD_STD_COST'] = std_cost
dfbod.at[bod_idx,'EOD_AMT_AVG_COST'] = eod_amt_avg_cost
dfbod.at[bod_idx,'EOD_TRX_AMT'] = eod_trx_amt

In [44]:
dfbod

,ITEM,DATE,TIME,BOD_QTY,BOD_AVG_COST,BOD_STD_COST,BOD_AMT_AVG_COST,BOD_TRX_AMT,BOD_BOM_COST,PO_QTY,...,MFG_QTY,MFG_AMT,COGS_QTY,COGS_AMT,EOD_QTY,EOD_AVG_COST,EOD_STD_COST,EOD_AMT_AVG_COST,EOD_TRX_AMT,EOD_BOM_COST
1,A,2020-05-02,00:00:01,10.0,10.0,10.2,100.0,100.0,10.0,0.0,...,0.0,0.0,0.0,0.0,3.0,9.81,10.2,29.43,29.45,0.0


In [45]:
# Update Excel of Transaction Records

In [46]:
df.to_excel(writer, sheet_name = 'TRX')

In [47]:
# Update the BOD/EOD record with PO. TRF, INV ADJ, MFG, COGS fields.

In [48]:
dftrx = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['GD_PO','GD_RETN']))]

In [49]:
dfbod.at[bod_idx,'PO_QTY'] = dftrx['QTY'].sum()
dfbod.at[bod_idx,'PO_AMT'] = dftrx['TRX_AMT'].sum()

In [50]:
dftrx = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['GD_OUT','GD_IN','RCP_OUT']))]

In [51]:
dfbod.at[bod_idx,'TRF_QTY'] = dftrx['QTY'].sum()
dfbod.at[bod_idx,'TRF_AMT'] = dftrx['TRX_AMT'].sum()

In [52]:
dftrx = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['INV_QTY_ADJ','INV_COST_ADJ']))]

In [53]:
dfbod.at[bod_idx,'ADJ_QTY'] = dftrx['QTY'].sum()
dfbod.at[bod_idx,'ADJ_AMT'] = dftrx['TRX_AMT'].sum()

In [54]:
dftrx = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['RCP_IN']))]

In [55]:
dfbod.at[bod_idx,'MFG_QTY'] = dftrx['QTY'].sum()
dfbod.at[bod_idx,'MFG_AMT'] = dftrx['TRX_AMT'].sum()

In [56]:
dftrx = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) & (df['DOC_TYPE'].isin(['COGS']))]

In [57]:
dfbod.at[bod_idx,'COGS_QTY'] = dftrx['QTY'].sum()
dfbod.at[bod_idx,'COGS_AMT'] = dftrx['TRX_AMT'].sum()

In [58]:
dfbod

,ITEM,DATE,TIME,BOD_QTY,BOD_AVG_COST,BOD_STD_COST,BOD_AMT_AVG_COST,BOD_TRX_AMT,BOD_BOM_COST,PO_QTY,...,MFG_QTY,MFG_AMT,COGS_QTY,COGS_AMT,EOD_QTY,EOD_AVG_COST,EOD_STD_COST,EOD_AMT_AVG_COST,EOD_TRX_AMT,EOD_BOM_COST
1,A,2020-05-02,00:00:01,10.0,10.0,10.2,100.0,100.0,10.0,2.0,...,0.0,0.0,-5.0,-49.05,3.0,9.81,10.2,29.43,29.45,0.0


In [59]:
# Update Excel of BOD/EOD Record

In [60]:
dfbod.to_excel(writer, sheet_name = 'DAY_WAC')
# writer.save()
# writer.close()

In [61]:
# Accounts Journal Processing

In [62]:
dfj = pd.read_excel (path, sheet_name='J_LOGIC')

In [63]:
dft = df[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) ]

In [64]:
dftj = pd.merge ( dft, dfj, left_on='DOC_TYPE', right_on='DOC_TYPE', how='left').copy()

In [65]:
dftj

,ITEM,DATE,TIME,DOC_TYPE,DOC_NO,RMKS,QTY,ACT_U_COST,USE_AVG_COST,AVG_COST,...,DR_ACCT,CR_ACCT,DR_COST_C,CR_COST_C,TRX_AMT_y,Remarks,DR,CR,DR_CC,CR_CC
0,A,2020-05-02,11:00:00,GD_PO,1,Goods Received,2,11.0,0.0,NAN,...,NaN,NaN,NaN,NaN,NaN,Goods In,Inventory,Vendor A/P Accrual,NaN,NaN
1,A,2020-05-02,11:30:00,GD_PO,2,Goods Received,1,12.0,0.0,NAN,...,NaN,NaN,NaN,NaN,NaN,Goods In,Inventory,Vendor A/P Accrual,NaN,NaN
2,A,2020-05-02,13:00:00,GD_RETN,3,Goods Returned for 2,-1,11.0,0.0,NAN,...,NaN,NaN,NaN,NaN,NaN,Goods Returned,Vendor A/P Accrual,Inventory,NaN,NaN
3,A,2020-05-02,12:00:00,GD_OUT,4,Goods Transferred Out,-2,NaN,1.0,10,...,NaN,NaN,NaN,NaN,NaN,Goods Transferred Out,Inventory,Inventory,OUT_CC,IN_CC
4,A,2020-05-02,12:00:00,GD_IN,5,Goods Transferred In,1,10.5,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Goods Transferred In,Inventory,Inventory,IN_CC,OUT_CC
5,A,2020-05-02,19:00:00,RCP_OUT,6,Ingredients used for Recipe DOC 107 ***,-2,10.0,1.0,10,...,NaN,NaN,NaN,NaN,NaN,Goods (ingredients) used in BOM,Inventory,Inventory,OUT_CC,IN_CC
6,A,2020-05-02,14:00:00,INV_QTY_ADJ,7,Inventory Qty Adjustment (e.g. write-off),-1,NaN,1.0,10,...,NaN,NaN,NaN,NaN,POS,Inventory Qty Adjustment Positive,Inventory,Expenses (Inventory Adjustment),NaN,NaN
7,A,2020-05-02,14:00:00,INV_QTY_ADJ,7,Inventory Qty Adjustment (e.g. write-off),-1,NaN,1.0,10,...,NaN,NaN,NaN,NaN,NEG,Inventory Qty Adjustment Negative,Expenses (Inventory Adjustment),Inventory,NaN,NaN
8,A,2020-05-02,15:00:00,INV_COST_ADJ,8,Inventory Cost Adjustment,0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,POS,Inventory Value Adjustment Positive,Inventory,Expenses (Inventory Adjustment),NaN,NaN
9,A,2020-05-02,15:00:00,INV_COST_ADJ,8,Inventory Cost Adjustment,0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NEG,Inventory Value Adjustment Negative,Expenses (Inventory Adjustment),Inventory,NaN,NaN


In [66]:
# Finance Reconciliation
# Inventory: Begining & End of Month record must reconcile with Balance Sheet Inventory
# PO: GD_IN & GD_RETURN records total must reconcile with Vendor A/P Accural Account
# COGS: COGS record must reconcile with COGS Account

In [67]:
# Cost Contol
#    Purchasing: Monitor Average Purchasing Cost vs. Standard Cost
#    Wastage, Leakage: Monitor Inventory Adjustement
#    Manufacturing/BOM: Monitor BOM Average Cost vs. BOM Standard Cost
#    Reconcile Standard Cost vs. Average Cost 
#             (with further comparison of Average Purchasing Cost Vs. Inventory Average Cost)
#
# For Buffet
#   MFG Cost = COGS & reconcile COGS/MFG Cost vs. Sales Amount
#   For Operational Cost Control, we should have a Real Time Expected Total MFG COST 
#                                (calculated by the expected (actual) number of buffet customers)

In [68]:
df

,ITEM,DATE,TIME,DOC_TYPE,DOC_NO,RMKS,QTY,ACT_U_COST,USE_AVG_COST,AVG_COST,ACT_AMT,AMT_AVG_COST,TRX_AMT,AMT_RD_VAR,DR_ACCT,CR_ACCT,DR_COST_C,CR_COST_C
0,A,2020-05-02,11:00:00,GD_PO,1,Goods Received,2,11.0,0.0,NAN,22.0,0.00,22.00,NaN,NaN,NaN,NaN,NaN
1,A,2020-05-02,11:30:00,GD_PO,2,Goods Received,1,12.0,0.0,NAN,12.0,0.00,12.00,NaN,NaN,NaN,NaN,NaN
2,A,2020-05-02,13:00:00,GD_RETN,3,Goods Returned for 2,-1,11.0,0.0,NAN,-11.0,0.00,-11.00,NaN,NaN,NaN,NaN,NaN
3,A,2020-05-02,12:00:00,GD_OUT,4,Goods Transferred Out,-2,NaN,1.0,10,NaN,-20.00,-20.00,NaN,NaN,NaN,NaN,NaN
4,A,2020-05-02,12:00:00,GD_IN,5,Goods Transferred In,1,10.5,1.0,NaN,10.5,0.00,10.50,NaN,NaN,NaN,NaN,NaN
5,A,2020-05-02,19:00:00,RCP_OUT,6,Ingredients used for Recipe DOC 107 ***,-2,10.0,1.0,10,NaN,-20.00,-20.00,NaN,NaN,NaN,NaN,NaN
6,A,2020-05-02,14:00:00,INV_QTY_ADJ,7,Inventory Qty Adjustment (e.g. write-off),-1,NaN,1.0,10,NaN,-10.00,-10.00,NaN,NaN,NaN,NaN,NaN
7,A,2020-05-02,15:00:00,INV_COST_ADJ,8,Inventory Cost Adjustment,0,NaN,0.0,NaN,-5.0,0.00,-5.00,NaN,NaN,NaN,NaN,NaN
8,A,2020-05-02,15:00:00,COGS,10,Goods sold,-5,NaN,NaN,9.81,NaN,-49.05,-49.05,NaN,NaN,NaN,NaN,NaN
9,B,2020-05-02,11:00:00,GD_PO,102,Goods Received,2,16.0,0.0,NaN,22.0,0.00,22.00,NaN,NaN,NaN,NaN,NaN


In [69]:
if not dftj.empty:
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) , 'DR_ACCT'] = dftj['DR']
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) , 'CR_ACCT'] = dftj['CR']
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) , 'DR_COST_C'] = dftj['DR_CC']
    df.loc[(df['DATE'] == bod_date) & (df['ITEM']== sel_item) , 'CR_COST_C'] = dftj['CR_CC']
    

In [70]:
df

,ITEM,DATE,TIME,DOC_TYPE,DOC_NO,RMKS,QTY,ACT_U_COST,USE_AVG_COST,AVG_COST,ACT_AMT,AMT_AVG_COST,TRX_AMT,AMT_RD_VAR,DR_ACCT,CR_ACCT,DR_COST_C,CR_COST_C
0,A,2020-05-02,11:00:00,GD_PO,1,Goods Received,2,11.0,0.0,NAN,22.0,0.00,22.00,NaN,Inventory,Vendor A/P Accrual,NaN,NaN
1,A,2020-05-02,11:30:00,GD_PO,2,Goods Received,1,12.0,0.0,NAN,12.0,0.00,12.00,NaN,Inventory,Vendor A/P Accrual,NaN,NaN
2,A,2020-05-02,13:00:00,GD_RETN,3,Goods Returned for 2,-1,11.0,0.0,NAN,-11.0,0.00,-11.00,NaN,Vendor A/P Accrual,Inventory,NaN,NaN
3,A,2020-05-02,12:00:00,GD_OUT,4,Goods Transferred Out,-2,NaN,1.0,10,NaN,-20.00,-20.00,NaN,Inventory,Inventory,OUT_CC,IN_CC
4,A,2020-05-02,12:00:00,GD_IN,5,Goods Transferred In,1,10.5,1.0,NaN,10.5,0.00,10.50,NaN,Inventory,Inventory,IN_CC,OUT_CC
5,A,2020-05-02,19:00:00,RCP_OUT,6,Ingredients used for Recipe DOC 107 ***,-2,10.0,1.0,10,NaN,-20.00,-20.00,NaN,Inventory,Inventory,OUT_CC,IN_CC
6,A,2020-05-02,14:00:00,INV_QTY_ADJ,7,Inventory Qty Adjustment (e.g. write-off),-1,NaN,1.0,10,NaN,-10.00,-10.00,NaN,Inventory,Expenses (Inventory Adjustment),NaN,NaN
7,A,2020-05-02,15:00:00,INV_COST_ADJ,8,Inventory Cost Adjustment,0,NaN,0.0,NaN,-5.0,0.00,-5.00,NaN,Expenses (Inventory Adjustment),Inventory,NaN,NaN
8,A,2020-05-02,15:00:00,COGS,10,Goods sold,-5,NaN,NaN,9.81,NaN,-49.05,-49.05,NaN,Inventory,Expenses (Inventory Adjustment),NaN,NaN
9,B,2020-05-02,11:00:00,GD_PO,102,Goods Received,2,16.0,0.0,NaN,22.0,0.00,22.00,NaN,NaN,NaN,NaN,NaN


In [71]:
df.to_excel(writer, sheet_name = 'TRX')
writer.save()
writer.close()